In [1]:
from typing import Optional, Dict, Any
from ray.tune.search.optuna import OptunaSearch
from ray.tune.search import ConcurrencyLimiter
from ray import tune
import time
from ray.air import session
import ray
import ray.tune.search.optuna.optuna_search

In [2]:
def define_by_run_func(trial:ray.tune.search.optuna.optuna_search._OptunaTrialSuggestCaptor) -> Optional[Dict[str, Any]]:
    """Define-by-run function to create the search space.

    Ensure no actual computation takes place here. That should go into
    the trainable passed to ``Tuner()`` (in this example, that's
    ``objective``).

    For more information, see https://optuna.readthedocs.io/en/stable\
    /tutorial/10_key_features/002_configurations.html

    This function should either return None or a dict with constant values.
    """

    activation = trial.suggest_categorical("activation", ["relu", "tanh"])

    # Define-by-run allows for conditional search spaces.
    if activation == "relu":
        trial.suggest_float("size", -100, 100)
        trial.suggest_float("si/ze/width", 0, 20)
        trial.suggest_float("size/height", -100, 100)
    else:
        trial.suggest_float("si/ze/width", -1, 21)
        trial.suggest_float("size", -101, 101)
        
    # Return all constants in a dictionary.
    return {"steps": 100}

In [3]:
searcher = OptunaSearch(define_by_run_func, metric="mean_loss", mode="min")
algo = ConcurrencyLimiter(searcher, max_concurrent=4)

[I 2023-07-01 20:05:22,228] A new study created in memory with name: optuna


In [4]:
def evaluate(step, width, height, activation):
    time.sleep(0.1)
    activation_boost = 10 if activation=="relu" else 0
    return (0.1 + width * step / 100) ** (-1) + height * 0.1 + activation_boost

In [5]:
def objective(config):
    for step in range(config["steps"]):
        score = evaluate(step, config["si"]["ze"]["width"], config["size"]["height"], config["activation"])
        session.report({"iterations": step, "mean_loss": score})

In [6]:
num_samples = 10
tuner = tune.Tuner(
    objective,
    tune_config=tune.TuneConfig(
        search_alg=searcher,
        num_samples=num_samples,
    ),
)
results = tuner.fit()

2023-07-01 20:05:24,588	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
2023-07-01 20:05:26,439	INFO tune.py:226 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `Tuner(...)`.


TypeError: Cannot unflatten dict due the key 'size/height' having a parent key 'size', which value is not of type <class 'dict'> (got <class 'float'>). Change the key names to resolve the conflict.

In [ ]:
print("Best hyperparameters for loss found were: ", results.get_best_result("mean_loss", "min").config)

Best hyperparameters for loss found were:  {'activation': 'tanh', 'si': {'ze': {'width': 17.423511459536684}}, 'size': {'height': -79.26626664441896}, 'steps': 100}


In [5]:
import torch
class ANN(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_layer, output_size):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layer = num_layer
        self.output_size = output_size
        layers = [[hidden_size, hidden_size] for i in range(num_layer)]
        layers[0][0] = input_size
        layers[-1][1] = output_size
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(layer[0], layer[1]) for layer in layers
        )
    
    def forward(self, x):
        x = self.layers(x)
        return x

In [6]:
ANN(3, 5, 3, 3)

TypeError: ANN.__init__.<locals>.<genexpr> is not a Module subclass